In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
sc = SparkContext(master="local", appName="PrimerRDD")

21/09/24 15:39:00 WARN Utils: Your hostname, DESKTOP-NVKHLIO resolves to a loopback address: 127.0.1.1; using 172.30.73.246 instead (on interface eth0)
21/09/24 15:39:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/24 15:39:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
sc

<SparkContext master=local appName=PrimerRDD>

In [3]:
rdd1 = sc.parallelize([1, 2, 3, 4, 5])

In [4]:
type(rdd1)

pyspark.rdd.RDD

In [5]:
rdd1.collect()

[1, 2, 3, 4, 5]

In [7]:
!ls

README.md  files  primerRDD.ipynb  primerasclases.ipynb


In [8]:
equiposOlimpicosRDD = sc.textFile("./files/paises.csv").map(lambda line: line.split(","))

In [9]:
equiposOlimpicosRDD.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

## Operaciones con los RDD

In [13]:
# trabaja normalmente con tuplas (tuplas son inmutables)
equiposOlimpicosRDD.map(lambda line: (line[2]) ).distinct().count()

231

In [15]:
# aqui utilizamos el groupBy para agrupar por pais y se usa con llave valor (key, value)
equiposOlimpicosRDD.map(lambda line: (line[2],line[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [16]:
equiposOlimpicosRDD.map(lambda line: (line[2],line[1])).groupByKey().mapValues(list).take(5)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise']),
 ('MEX',
  ['A North American Team',
   'Acipactli',
   'Chamukina',
   'Mexico',
   'Mexico-1',
   'Mexico-2',
   'Nausikaa 4',
   'Tlaloc',
   'Xolotl']),
 ('ARG',
  ['Acturus',
   'Antares',
   'Arcturus',
   'Ardilla',
   'Argentina',
   'Argentina-1',
   'Argentina-2',
   'Blue Red',
   'Covunco III',
   'Cupidon III',
   'Djinn',
   'Gullvinge',
   'Matrero II',
   'Mizar',
   'Pampero',
   'Rampage',
   'Tango',
   'Wiking']),
 ('AFG', ['Afghanistan'])]

In [18]:
equiposArgentinos = equiposOlimpicosRDD.filter(lambda line: "ARG" in line)
equiposArgentinos.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

El colect no es bueno usarlo a la hora de llamar datos por que trae todos los registros que pueda haber en el conjunto de clusteres

por ejemplo si necesitamos contar un rdd si este esmuy grande seria una operacion peligrosa de realizar por lo que necesitamos una funcion coun aproximadamente

In [20]:
equiposOlimpicosRDD.countApprox(20)

1185

In [23]:
deportistasOlimpicosRDD = sc.textFile("./files/deportista.csv").map(lambda line: line.split(",")) \
    .union(sc.textFile("./files/deportista2.csv").map(lambda line: line.split(",")))

In [24]:
deportistasOlimpicosRDD.count()

135572

In [25]:
deportistasOlimpicosRDD.top(2)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967']]

In [44]:
df = deportistasOlimpicosRDD.toDF(['deportista_id','nombre', 'genero', 'edad', 'altura','peso','equipo_id'])
df.show()

AttributeError: 'RDD' object has no attribute 'toDF'

In [34]:
equiposJugadores = deportistasOlimpicosRDD.map(lambda line: (line[-1],line[:-1])) \
    .join(equiposOlimpicosRDD.map(lambda line: (line[0],line[2])))

In [29]:
resultadosRDD = sc.textFile("./files/resultados.csv").map(lambda line: line.split(","))

In [31]:
resultadosGanadoresRDD = resultadosRDD.filter(lambda line: 'NA' not in line)
resultadosGanadoresRDD.take(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['4', 'Gold', '4', '2', '4'],
 ['38', 'Bronze', '15', '7', '19'],
 ['39', 'Bronze', '15', '7', '20'],
 ['41', 'Bronze', '16', '50', '14']]

In [35]:
equiposJugadoresMedallas = resultadosGanadoresRDD.map(lambda line: (line[0],line[1])) \

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN'))]

In [ ]:
df['suma'] = df.A + df.B

In [47]:
rdd = sc.parallelize(
    (("first", [2.0, 1.0, 2.1, 5.4]),
    ("test", [1.5, 0.5, 0.9, 3.7]),
    ("choose", [8.0, 2.9, 9.1, 2.5]))
)

In [48]:
rdd.take(5)

[('first', [2.0, 1.0, 2.1, 5.4]),
 ('test', [1.5, 0.5, 0.9, 3.7]),
 ('choose', [8.0, 2.9, 9.1, 2.5])]

In [52]:
data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]
df = sc.createDataFrame(data=data, schema = columns)

AttributeError: 'SparkContext' object has no attribute 'createDataFrame'